# Libraries
The libraries that will be used for preprocessing.

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np

# Data Analysis for Errors

The data must be analysed for errors so that they can be fixed before processing.

In [2]:
file = r"Datasets\winequalityN.csv"

df = pd.read_csv(file)

df

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,white,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,white,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,red,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
6493,red,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,NaN,11.2,6
6494,red,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
6495,red,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


### Data Types

Identify the data types of each column.

In [3]:
print("Data Types")
print()
print(df.dtypes)

Data Types

type                     object
fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object


### Missing Values

Exploring the dataset for null values, if they exist they can make processing difficult or impossible.

In [4]:
print("Null Value Analysis")
print()
null_values = df.isnull().sum()
print(null_values[null_values > 0])

Null Value Analysis

fixed acidity       10
volatile acidity     8
citric acid          3
residual sugar       2
chlorides            2
pH                   9
sulphates            4
dtype: int64


### Outlier Count

Identify potetial outliers in numeric columns by checking values beyond the 1st and 99th percentile.

In [5]:
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
outlier_counts = []

for col in numeric_cols:
    outliers = ((df[col] < df[col].quantile(0.01)) | (df[col] > df[col].quantile(0.99))).sum()
    outlier_counts.append((col, outliers))
outlier_counts = pd.DataFrame(outlier_counts, columns=["Column", "Outliers"])

print("Outlier Count")
print()
print(outlier_counts)

Outlier Count

                  Column  Outliers
0          fixed acidity       123
1       volatile acidity        96
2            citric acid        29
3         residual sugar        97
4              chlorides       123
5    free sulfur dioxide       126
6   total sulfur dioxide       125
7                density       122
8                     pH       122
9              sulphates       115
10               alcohol        95
11               quality        35


### Summary Statistics

A statistical summary, calculating values such as the mean and identifying values such as the max.

In [6]:
print("Summary")
print()
print(df.describe(include='all'))

Summary

         type  fixed acidity  volatile acidity  citric acid  residual sugar  \
count    6497    6487.000000       6489.000000  6494.000000     6495.000000   
unique      2            NaN               NaN          NaN             NaN   
top     white            NaN               NaN          NaN             NaN   
freq     4898            NaN               NaN          NaN             NaN   
mean      NaN       7.216579          0.339691     0.318722        5.444326   
std       NaN       1.296750          0.164649     0.145265        4.758125   
min       NaN       3.800000          0.080000     0.000000        0.600000   
25%       NaN       6.400000          0.230000     0.250000        1.800000   
50%       NaN       7.000000          0.290000     0.310000        3.000000   
75%       NaN       7.700000          0.400000     0.390000        8.100000   
max       NaN      15.900000          1.580000     1.660000       65.800000   

          chlorides  free sulfur dioxide  

# Data Cleaning

Before commencing, it would be best to duplicate the dataset so that if it is necessary the original dataset is always available

In [7]:
df2 = df.copy()

df2.to_csv("Datasets/WineQualityDuplicate.csv", index=False)

df2 = pd.read_csv(r"Datasets\\WineQualityDuplicate.csv")

The "type" column should be replaced with an encoded version.

In [8]:
label_encoder = LabelEncoder()

df2['type'] = label_encoder.fit_transform(df['type'])

print(df2[['type']].head())

   type
0     1
1     1
2     1
3     1
4     1


The next step chosen is to identify drop the rows with missing values as we have several thousand samples.

Below, the null value analysis should have the previous columns with null values.

In [9]:
print("Null Value Analysis")
print()
null_values = df2.isnull().sum()
print(null_values[null_values > 0])

Null Value Analysis

fixed acidity       10
volatile acidity     8
citric acid          3
residual sugar       2
chlorides            2
pH                   9
sulphates            4
dtype: int64


The rows are with null values are dropped,

In [10]:
cleaned = df2.dropna()

cleaned.to_csv(r"Datasets\\WineQualityDuplicate.csv", index=False)

df2 = pd.read_csv("Datasets\\WineQualityDuplicate.csv")

and it can now be found that there are no columns with null values.

In [11]:
print("Null Value Analysis")
print()
null_values = df2.isnull().sum()
print(null_values[null_values > 0])

Null Value Analysis

Series([], dtype: int64)


Now we can effectively drop the rows with outliers by eliminating the values in the upper and lower margin.

In [12]:
numeric_cols = df2.select_dtypes(include=['float64', 'int64']).columns

outlier_mask = pd.Series(True, index=df2.index)

for col in numeric_cols:
    lower_bound = df2[col].quantile(0.10)
    upper_bound = df2[col].quantile(0.90)
    outlier_mask &= (df2[col] >= lower_bound) & (df2[col] <= upper_bound)

cleaned = df2[outlier_mask]

cleaned.to_csv(r"Datasets\\WineQualityDuplicate.csv", index=False)

df2 = pd.read_csv(r"Datasets\\WineQualityDuplicate.csv")

# Feature Enginering